In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import scipy.stats as vale
import seaborn as sns
from sklearn import linear_model
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import statsmodels as stats



In [ ]:
calendario= pd.read_csv("calendar.csv")
cat_art=pd.read_csv("Datos_limpios/cat_articulos.csv",index_col=0)
cat_st=pd.read_csv("Datos_limpios/cat_tiendas.csv",index_col=0)
ventas_trans= pd.read_csv("Datos_limpios/ventas_transacc.csv",index_col=0)
submision=pd.read_csv("sample_submission.csv")
precio_venta=pd.read_csv("sell_prices.csv")


## Realiza un diagrama de Pareto de las ventas globales por tienda. ¿Existe mucha diferencia en el rendimiento entre tiendas con respecto a la venta?

In [ ]:
#print(calendario)

#print(ventas_trans)
#print(cat_art)
print(cat_st)
precio_venta_num= precio_venta.merge(cat_art[["item_id","num_item_id"]],on= "item_id")
precio_venta_num=precio_venta_num.merge(cat_st[["store_id","num_store"]],on="store_id")
precio_cantidad= ventas_trans.merge(calendario[["wm_yr_wk","d"]],on="d")


precio_cantidad=precio_cantidad.merge(precio_venta_num[["num_store","num_item_id","wm_yr_wk","sell_price"]],on=["num_store","num_item_id","wm_yr_wk"])
#print(precio_cantidad)

precio_cantidad["Revenue"]=precio_cantidad["sell_price"]*precio_cantidad["sales"]
print(precio_cantidad)

In [ ]:
pareto_ventas= precio_cantidad.groupby(["num_store"]).sum().reset_index()
pareto_ventas=pareto_ventas.merge(cat_st,on="num_store")
pareto_ventas_cantidad=pareto_ventas[["store_id","sales",]].sort_values("sales",ascending=False)
pareto_ventas_revenue=pareto_ventas[["store_id","Revenue"]].sort_values("Revenue",ascending= False)
total_cantidad=pareto_ventas_cantidad["sales"].sum()
total_revenue=pareto_ventas_revenue["Revenue"].sum()
pareto_ventas_cantidad["%"]=pareto_ventas_cantidad["sales"]/total_cantidad
pareto_ventas_cantidad["ac"]=pareto_ventas_cantidad["%"].cumsum()
#print(pareto_ventas)
print(pareto_ventas_cantidad)
pareto_ventas_cantidad.plot.bar(x="store_id", y= "%")

print(pareto_ventas)

In [ ]:
pareto_ventas_revenue["%"]=pareto_ventas_revenue["Revenue"]/total_revenue
pareto_ventas_revenue["ac"]=pareto_ventas_revenue["%"].cumsum()
print(pareto_ventas_revenue)
pareto_ventas_revenue.plot.bar(x="store_id",y="%")

In [ ]:
estado_sales=pareto_ventas.groupby(["state_id"])["sales"].sum().reset_index().sort_values("sales",ascending=False)
estado_sales["%"]=estado_sales["sales"]/total_cantidad
estado_sales["ac"]=estado_sales["%"].cumsum()
estado_revenue=pareto_ventas.groupby(["state_id"])["Revenue"].sum().reset_index().sort_values("Revenue",ascending=False)
estado_revenue["%"]=estado_revenue["Revenue"]/total_revenue
estado_revenue["ac"]= estado_revenue["%"].cumsum()

print(estado_sales)
estado_sales.plot.bar(x="state_id", y= "%")
print(estado_revenue)
estado_revenue.plot.bar(x="state_id", y= "%")

Realiza un diagrama de Pareto de las ventas globales por categoría de productos, y de las ventas globales por departamento.
Posteriormente, realiza el Pareto de las categorías y de los departamentos por estado (California, Texas, Wisconsin). ¿Varía mucho qué tipo de productos se venden por estado?

In [ ]:
#print(precio_cantidad)
#print(cat_art)
pareto_gen= precio_cantidad[["num_item_id","sales","num_store"]].merge(cat_art[["num_item_id","dept_id","cat_id"]],on="num_item_id")
pareto_gen=pareto_gen.merge(cat_st[["num_store","state_id"]],on="num_store")
pareto_cat=pareto_gen.groupby(["dept_id"]).sum()
print(pareto_gen)
pareto_cat=pareto_gen.groupby(["dept_id"]).sum().reset_index().sort_values("sales",ascending=False)
pareto_cat["%"]= pareto_cat["sales"]/total_cantidad
pareto_cat["ac"]=pareto_cat["%"].cumsum()
#print(pareto_cat)
pareto_dept=pareto_gen.groupby(["cat_id"]).sum().reset_index().sort_values("sales",ascending=False)
pareto_dept["%"]= pareto_dept["sales"]/total_cantidad
pareto_dept["ac"]=pareto_dept["%"].cumsum()
#print(pareto_dept)
#dept=cat_art.groupby(["dept_id"]).count()
                     
#print(dept)
fig,axs=plt.subplots(nrows=1,ncols=2)

pareto_dept.plot.bar(x="cat_id", y= "%",title="pareto dept",ax=axs[0])
pareto_cat.plot.bar(x="dept_id", y= "%",title="pareto cat",ax=axs[1])
fig.suptitle("estadistica global",fontsize=16)
plt.show()



In [ ]:
states=pareto_gen["state_id"].unique()
from matplotlib import rcParams
rcParams["figure.figsize"]=(16,8)
for estado in states :
    lista_estado=pareto_gen[pareto_gen["state_id"]==estado]
    pareto_state=lista_estado[["sales","state_id","num_item_id","dept_id"]].groupby(["dept_id","state_id"]).sum().reset_index().sort_values("sales",ascending=False)
    pareto_state["%"]=pareto_state["sales"]/pareto_state["sales"].sum()
    pareto_state["ac"]=pareto_state["%"].cumsum()
    pareto_state_cat=lista_estado[["sales","state_id","num_item_id","cat_id"]].groupby(["cat_id","state_id"]).sum().reset_index().sort_values("sales",ascending=False)
    pareto_state_cat["%"]=pareto_state_cat["sales"]/pareto_state_cat["sales"].sum()
    pareto_state_cat["ac"]=pareto_state_cat["%"].cumsum()
    print(pareto_state)
    print(pareto_state_cat)
    fig,axs=plt.subplots(nrows=1,ncols=2)

    pareto_state.plot.bar(x="dept_id", y= "%",title="pareto state",ax=axs[0])
    pareto_state_cat.plot.bar(x="cat_id", y= "%",title="pareto state",ax=axs[1])
    fig.suptitle("estadistica" + estado ,fontsize=16)
    plt.show()

Genera un histograma del dato de las ventas, al nivel en el que se encuentren originalmente los datos. ¿Qué tan grande es este valor? ¿Esto da evidencia de que las ventas, al nivel en el que se encuentra, son intermitentes o son estables?


In [ ]:
print(precio_cantidad)
hist=precio_cantidad.groupby(["num_item_id"]).sum()
#hist["sales"].plot.hist(log=True)
print(hist)
precio_cantidad["sales"].plot.hist(log=True)

print(precio_cantidad.groupby("sales")["sell_price"].count())

es intermitente debido a que la mayoria de las observaciones de venta son 0 aun cuando hay datos de precio valido , se concluye que tenemos series de tiempo intermitente

## Muestrea 3 productos de la categoría hobbies, 3 productos de la categoría Foods, y 3 productos de la categoría Household. Utilizando la venta nacional, revisa el comportamiento de estos con gráficas de series de tiempo. ¿Existe un comportamiento diferente por categoría? ¿Qué tan altas son las ventas de estos productos?

In [ ]:
np.random.seed(10)
print(np.random.normal(10,5,20))

In [ ]:
productos_hobbies=cat_art[cat_art["cat_id"]=="HOBBIES"].sample(3,random_state=532)["num_item_id"]
productos_households=cat_art[cat_art["cat_id"]=="HOUSEHOLD"].sample(3,random_state=532)["num_item_id"]
productos_foods=cat_art[cat_art["cat_id"]=="FOODS"].sample(3,random_state=532)["num_item_id"]
print(productos_hobbies)
print(productos_households)
print(productos_foods)

In [ ]:
#print(precio_cantidad)
serie_global=precio_cantidad.groupby(["num_item_id","d"])["sales"].sum().reset_index()
serie_global=serie_global.merge(calendario[["d","date"]], on="d")
serie_hobbies=serie_global[serie_global["num_item_id"].isin(productos_hobbies)]
serie_hobbies["num_item_id"]=serie_hobbies["num_item_id"].astype(str)

serie_hobbies["str_item_id"]=["producto"+str(id) for id in serie_hobbies["num_item_id"]]

sns.lineplot(x="date",y="sales",hue="str_item_id",data=serie_hobbies)
print(serie_hobbies)




In [ ]:
serie_households=serie_global[serie_global["num_item_id"].isin(productos_households)]
serie_households["num_item_id"]=serie_households["num_item_id"].astype(str)

serie_households["str_item_id"]=["producto"+str(id) for id in serie_households["num_item_id"]]

sns.lineplot(x="date",y="sales",hue="str_item_id",data=serie_households)
print(serie_households)



In [ ]:
serie_foods=serie_global[serie_global["num_item_id"].isin(productos_foods)]
serie_foods["num_item_id"]=serie_foods["num_item_id"].astype(str)

serie_foods["str_item_id"]=["producto"+str(id) for id in serie_foods["num_item_id"]]

sns.lineplot(x="date",y="sales",hue="str_item_id",data=serie_foods)
print(serie_foods)

## Realiza un pareto por Producto a nivel nacional. ¿Cómo se comporta esta distribución?

In [ ]:
#print(precio_cantidad)

pareto_global= precio_cantidad.groupby(["num_item_id"])["sales"].sum().reset_index().sort_values("sales",ascending=False)
pareto_global["%"]=pareto_global["sales"]/total_cantidad
pareto_global["ac"]=pareto_global["%"].cumsum()
pareto_global.reset_index()["ac"].plot()
print(pareto_global["ac"])

# Genera un diagrama de cajas, por departamento, de la variable de precio. ¿Existe una variación importante por departamento?

In [ ]:
#print(precio_venta)
#print(cat_art)

diagrama_dep= precio_venta[["item_id","sell_price"]].merge(cat_art[["item_id","dept_id"]],on="item_id")
#sns.boxplot(x="dept_id",y="sell_price",data=diagrama_dep)

diagrama_prom= diagrama_dep.groupby(["item_id","dept_id"]).mean().reset_index()
print(diagrama_prom)
print(diagrama_dep)
sns.violinplot(x="dept_id",y="sell_price",data=diagrama_prom)


# Para cada departamento, genera una gráfica de serie de tiempo (con frecuencia mensual) del precio promedio. ¿De casualidad el precio ha incrementado con respecto al tiempo?

In [ ]:
#print(cat_art)
#print(precio_venta)
#print(calendario)

info_ventas= calendario[["month","year","date","wm_yr_wk"]].merge(precio_venta[["item_id","wm_yr_wk","sell_price"]],on="wm_yr_wk")
info_ventas= info_ventas.merge(cat_art[["item_id","dept_id"]],on="item_id")


info_ventas= info_ventas.groupby(["year","month","dept_id","item_id"])["sell_price"].mean().reset_index()
info_ventas["time"]= info_ventas["year"]+(info_ventas["month"]-1)/12
info_ventas["str_time"]=info_ventas["time"].astype(str)
from matplotlib import rcParams

rcParams["figure.figsize"]=(25,8)
sns.boxplot(x="str_time",y="sell_price",data=info_ventas[info_ventas["dept_id"]=="FOODS_1"])
print(info_ventas)


In [ ]:
dept_id= info_ventas["dept_id"].unique()
print(dept_id)
rcParams["figure.figsize"]=(25,8)
for departamento in dept_id:
    
    sns.boxplot(x="str_time",y="sell_price",data=info_ventas[info_ventas["dept_id"]== departamento]).set_title("precios "+ departamento)
    plt.show()

In [ ]:
#info_foods1=info_ventas[info_ventas["dept_id"]=="FOODS_1"]
#info_foods1["time"]= info_foods1["year"]+(info_foods1["month"]-1)/12
#print(info_foods1)
#info_foods1.plot(x="time",y="sell_price")

# ¿Cuáles son los eventos que marca como importante la base de datos? ¿Hay alguna manera de revisar si tienen efecto importante en las ventas a nivel nacional?

In [ ]:
print(calendario)

In [ ]:
#print(calendario["event_name_1"].unique())
#print(calendario["event_name_2"].unique())


calendario_puente=calendario[(~calendario["event_name_1"].isna())]
calendario_puente=calendario_puente[["d","event_name_1","event_type_1","event_name_2","event_type_2"]].merge(ventas_trans[["d","sales"]],on="d")
calendario_puente=calendario_puente.fillna("NA")
calendario_puente=calendario_puente.groupby(["d","event_name_1","event_type_1","event_name_2","event_type_2"]).sum().reset_index()
print(calendario_puente)
#print(ventas_trans)

In [ ]:
event_name=[]
event_type=[]
sales=[]

for index, row in calendario_puente.iterrows():
    
    event_name.append(row["event_name_1"])
    event_type.append(row["event_type_1"])
    sales.append(row["sales"])
    
    if row["event_name_2"]!="NA":
        event_name.append(row["event_name_1"])
        event_type.append(row["event_type_2"])
        sales.append(row["sales"])
        
calendario_puente_final= pd.DataFrame({"event_name": event_name, "event_type": event_type,"sales":sales})   
ventas_evento=calendario_puente_final.groupby(["event_name"]).mean().reset_index()

ventas_type=calendario_puente_final.groupby(["event_type"]).mean().reset_index()

#print(ventas_type)
total_ventas=ventas_trans["sales"].sum()

promedio_dia=total_ventas/1913

ventas_evento["effect"]=ventas_evento["sales"]/promedio_dia
ventas_type["effect"]=ventas_type["sales"]/promedio_dia
print(ventas_evento)
print(ventas_type)

